In [ ]:
# For labels for real guitar data
import os
import shutil
import numpy as np
import pyfftw
import scipy
from scipy.io import wavfile as wav
import matplotlib.pyplot as plt
import ffmpeg
import PIL
from PIL import Image
import spectrogramFuncs as spec
import midiCreationFuncs as midiMagic
import makeDataset as md
import pretty_midi as pm

In [ ]:
# Guitar
#fNames = ['4AscFTeleClean', '4AscGLPClean', '8AscFTeleClean', '8AscGLPClean']
#midiFName = ['4Asc']*5 + ['8Asc']*5
#other = ['', 'CleanCitrus', 'Blues74', 'LittleJimisWing', 'JazzReverb']*2
#noteLengths = [0.5]*5 + [0.25]*5

# For basic guitar soundfonts
fNames = ['4Asc']
midiFName = ['4Asc']*7
other = ['Clean', 'Jazz', 'Overdrive', 'Funk', 'Distortion', 'GLPClean', 'FTeleClean']
noteLengths = [0.5]*7

directory = 'Guitar/Hop Experiments Single Note Ascensions'

In [ ]:
# Trumpet
fNames = ['4Asc']#'4AscNoteX4']#, '8AscNoteX4']
midiFName = ['4Asc']*3#'4AscNoteX4']*3# + ['8AscNoteX4']*3
noteLengths = [0.5]*3# + [0.25]*3
other = ['Clean', 'Expr', 'Pap']*3
directory = 'Trumpet'

In [ ]:
# Piano
fNames = ['4Asc']
midiFName = ['4Asc']*8
noteLengths = [0.5]*8
other = ['ChateauGrand', 'YamahaC5', 'LargeConcertGrand', 'Upright', 'GrandPiano', 'BrightGrandPiano', 'MellowGrandPiano','ElectricGrand']
directory = 'Piano'

In [ ]:
# Combined
fNames = ['4Asc']#'4AscGLPCleanTrump', '4AscFndTeleTrump']#, '4AscGLPCleanCitrusVSCO2']#['4AscGLPCleanPapTrump', '4AscFndTelePapTrump', 
midiFName = ['4Asc']
noteLengths = [0.5]
other = ['CleanTrump']
directory = 'Guitar+Trumpet'

In [ ]:
# Checking stuff
path = 'TrainingRawData(120)/' + directory + "/" + midiFName[0] + ".mid"
info = pm.PrettyMIDI(path, initial_tempo=120)
pm.program_to_instrument_name(info.instruments[1].program)

In [ ]:
# 2 Insts
labels = [[],[]]
datas = np.ndarray(shape=(0,512,128,3), dtype='float16', order='C')
fwindow  = 1024
hopLength = 512
directory = directory
#print(labels[0])
#print(len(labels[0]))
#print(np.argmax(labels[0]))


In [ ]:
# Two Label Sys
repeat = 0
while repeat < 1:
    for filo in fNames:
        for midiFile, noteLength, post in zip(midiFName, noteLengths, other):
            if midiFile in filo: # Change depending on instrument inputting (either == or in)
                print(filo, midiFile, post)
                guit, trump = midiMagic.create_inst_label('TrainingRawData(120)/' + directory + "/" + midiFile + ".mid", noteLength, (fwindow*2)/32768)

                spec.create_spectrogram_set_from_wav_file('TrainingRawData(120)/' + directory + "/" + filo + post, hopLength = hopLength, fwindow = fwindow, reset=False)
                
                data = md.spectrograms_to_arrays('TrainingRawData(120)/' + directory + '/' + filo + post + 'Spectrograms' + 'F_' + str(fwindow) + 'H_' + str(hopLength))

                # Add labels for nothing, let = label 129
                nothingLength = len(data)-len(guit)
                while nothingLength > 0:
                    guit.append([0]*128 + [1])
                    trump.append([0]*128 + [1])
                    nothingLength -= 1
                #data = data[:len(label_0)]
                
                labels[0].extend(guit)
                labels[1].extend(trump)
                datas = np.concatenate((datas, data), axis = 0)
                '''
                # Duplicate Data (for guitar, trump only)
                labels[0].extend(guit)
                labels[1].extend(trump)
                datas = np.concatenate((datas,data), axis = 0)'''
            else: 
                print('skipped')
                continue 
    repeat += 1

print(np.shape(labels))
#print(np.shape(datas))

In [ ]:
labels = np.array(labels)
md.create_two_label_dataset('TrainingDatasets/Guitar_Trump/','GTSetF1024_H512_Take_3_Size_37445_4AscOnly', datas, labels)

In [ ]:
for i,j in zip(labels[0], labels[1]):
    print(np.argmax(i), np.argmax(j))
print(np.shape(labels))
print(np.shape(datas))

In [ ]:
# Duplicate Data
labels[0].extend(labels[0])
labels[1].extend(labels[1])
datas = np.concatenate((datas,datas), axis = 0)
print(np.shape(labels))
print(np.shape(datas))

In [ ]:
# 1 Inst
labels = np.ndarray(shape=(0,129), dtype='float32', order='C')
datas = np.ndarray(shape=(0,512,128,3), dtype='float32', order='C')
fwindow  = 4096
hopLength = 512
directory = directory
label_ins = 'Trumpet'

In [ ]:
# 1 Label Sys
i = 0
repeat = 0
while repeat < 1:
    for filo in fNames:
        for midiFile, noteLength, post in zip(midiFName, noteLengths, other):
            if midiFile in filo: # Change depending on instrument inputting (either == or in)
                print(filo, midiFile, post)
                inst = midiMagic.create_sole_inst_label('TrainingRawData(120)/' + directory + "/" + midiFile + ".mid", noteLength, (fwindow*2)/32768, label_ins)
                #if i < 5:
                #spec.create_spectrogram_set_from_wav_file('TrainingRawData(120)/' + directory + "/" + filo + post, hopLength = hopLength, fwindow = fwindow)
                #    i += 1
                data = md.spectrograms_to_arrays('TrainingRawData(120)/' + directory + '/' + filo + post + 'Spectrograms' + 'F_' + str(fwindow) + 'H_' + str(hopLength))
                # Add labels for nothing, let = label 129
                nothingLength = len(data)-len(inst)
                while nothingLength > 0:
                    inst.append([0]*128 + [1])
                    nothingLength -= 1
                #data = data[:len(label_0)]
                
                labels = np.concatenate((labels,inst), axis = 0)
                datas = np.concatenate((datas, data), axis = 0)
                '''
                # Duplicate Data (for guitar, trump only)
                labels[0].extend(guit)
                labels[1].extend(trump)
                datas = np.concatenate((datas,data), axis = 0)'''
            else: 
                print('skipped')
                continue 
    repeat += 1
print('--- Complete---')

In [ ]:
for i in range(len(labels)):
    print(np.argmax(labels[i]),)
print(np.shape(labels))
print(np.shape(datas))

In [ ]:
labels = np.array(labels)
md.create_one_label_dataset('TrainingDatasets/Trumpet1Inst/','TrumpetSoundFontSetF4096_H512_Combined_Take_2_Size_9361_4AscOnly', datas, labels) 